#### From this issue: https://github.com/luispintoc/Linking-Writing/issues/4

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft
from skimage import feature
import scipy
import numpy as np
import pandas as pd
from scipy.stats import linregress
from numba import njit, prange

In [2]:
df = pd.read_csv('../kaggle feat_eng/train_logs_corrected.csv')

In [32]:
df[(df['down_event']=='Delete')&(df['text_change']=='q')]

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
33676,00f0737e,138,79477,79716,6.638889e-08,Remove/Cut,Delete,Delete,q,4,1
34075,00f0737e,537,247208,247352,4.000000e-08,Remove/Cut,Delete,Delete,q,270,43
34077,00f0737e,539,249066,249227,4.472222e-08,Remove/Cut,Delete,Delete,q,271,43
34888,00f0737e,1350,638426,638586,4.444444e-08,Remove/Cut,Delete,Delete,q,856,139
35931,00f0737e,2393,1794223,1794236,3.611111e-09,Remove/Cut,Delete,Delete,q,1226,251
...,...,...,...,...,...,...,...,...,...,...,...
8383207,ff9a11ae,2979,1419848,1419908,1.666667e-08,Remove/Cut,Delete,Delete,q,1577,265
8383208,ff9a11ae,2980,1420226,1420294,1.888889e-08,Remove/Cut,Delete,Delete,q,1577,265
8383209,ff9a11ae,2981,1420409,1420477,1.888889e-08,Remove/Cut,Delete,Delete,q,1577,265
8383210,ff9a11ae,2982,1420628,1420665,1.027778e-08,Remove/Cut,Delete,Delete,q,1577,265


In [22]:
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified', 'Clear', 'PageUp', 'PageDown']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']

In [27]:
def get_avg_down_event_per_minute(df, events):

    # Convert max down_time from milliseconds to minutes for each id
    max_down_time = df.groupby('id')['down_time'].max() / (1000 * 60)

    # Count number of each unique down_event for each id
    event_count = df[df['down_event'].isin(events)].groupby(['id', 'down_event']).size().reset_index(name='counts')

    # Calculate average number of each unique down_event per minute
    event_count['avg_event_per_minute'] = event_count['counts'] / event_count['id'].map(max_down_time)

    # Pivot the DataFrame
    pivot_df = event_count.pivot(index='id', columns='down_event', values='avg_event_per_minute')

    # Fill NaN values with 0
    pivot_df = pivot_df.fillna(0)

    return pivot_df

In [28]:
get_avg_down_event_per_minute(df, events)

down_event,',",",.,ArrowDown,ArrowLeft,ArrowRight,ArrowUp,Backspace,CapsLock,Clear,Delete,Enter,Leftclick,PageDown,PageUp,Shift,Space,Unidentified,q
id,,,,,,,,,,,,,,,,,,,
001519c8,0.099896,0.399583,0.699271,0.000000,0.066597,0.066597,0.000000,13.885520,0.000000,0.0,0.0,0.133194,3.063472,0.0,0.0,0.899062,11.887604,0.0,53.910450
0022f953,0.100624,0.704366,0.503119,0.100624,1.643521,1.542898,0.067083,8.720725,0.000000,0.0,0.0,0.201248,1.878310,0.0,0.0,3.253501,13.114629,0.0,49.976465
0042269b,0.000000,0.779124,0.711374,0.000000,0.000000,0.203250,0.000000,14.871114,0.000000,0.0,0.0,0.575875,4.369872,0.0,0.0,1.321124,18.698986,0.0,98.372928
0059420b,0.085446,0.128169,0.555400,0.000000,0.000000,0.000000,0.000000,6.493904,0.085446,0.0,0.0,0.128169,0.769015,0.0,0.0,2.905168,10.381702,0.0,44.346530
0075873a,0.613574,0.866223,0.830130,0.000000,0.000000,0.000000,0.000000,18.659881,0.000000,0.0,0.0,0.360926,1.191056,0.0,0.0,1.407612,11.694007,0.0,55.618718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffb8c745,0.234430,1.071679,1.440069,0.000000,0.000000,0.000000,0.000000,32.150374,0.000000,0.0,0.0,0.234430,0.803759,0.0,0.0,5.492356,21.801973,0.0,95.245484
ffbef7e5,0.266797,0.800390,1.033836,0.000000,0.000000,0.000000,0.000000,2.000974,0.000000,0.0,0.0,0.400195,1.200584,0.0,0.0,3.535054,14.907255,0.0,62.497082
ffccd6fd,0.000000,0.061247,0.153118,0.888085,1.623051,0.888085,0.000000,2.694877,0.183742,0.0,0.0,0.367483,0.275612,0.0,0.0,0.000000,56.990527,0.0,29.674272


In [38]:


# Define a function to process the DataFrame
def calculate_deletions_per_min(df):

    def calc(df, event_name):
        # Convert down_time from milliseconds to seconds
        df['down_time_sec'] = df['down_time'] / 1000
        # Convert down_time_sec to datetime
        df['down_time_sec'] = pd.to_datetime(df['down_time_sec'], unit='s')
        # Set down_time_sec as index
        df.set_index('down_time_sec', inplace=True)
        # Calculate the length of deletions
        df['deletion_length'] = df['text_change'].str.len()
        # Resample to 1 minute intervals and sum deletion length
        df_sum = df.groupby('id').resample('1T')['deletion_length'].sum()
        # Calculate the average of the sums for each ID
        df_avg = df_sum.groupby('id').mean()
        # Reset index
        df_avg = df_avg.reset_index()
        # Rename the column
        df_avg.rename(columns={'deletion_length': f'{event_name}'}, inplace=True)

        return df_avg

    # Filter the DataFrame for 'Delete' down_event and 'q' text_change
    df_delete = df[(df['down_event']=='Delete')&(df['text_change']=='q')]
    
    # Repeat the same steps for 'Backspace'
    df_backspace = df[(df['down_event']=='Backspace')&(df['text_change']=='q')]

    temp1 = calc(df_delete, 'Delete')
    temp2 = calc(df_backspace, 'Backspace')

    # Merge the DataFrames
    df_avg = temp1.merge(temp2, on='id')

    return df_avg

# Process the DataFrames
df_del = calculate_deletions_per_min(df)

print(df_del)

C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\2120278322.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['down_time_sec'] = df['down_time'] / 1000
C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\2120278322.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['down_time_sec'] = pd.to_datetime(df['down_time_sec'], unit='s')
C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\2120278322.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

           id    Delete  Backspace
0    00f0737e  0.172414   6.689655
1    01963e20  7.900000  38.950000
2    02a41d1a  5.352941   9.846154
3    02cf6a52  2.200000   5.909091
4    031c0c58  3.000000  10.173913
..        ...       ...        ...
379  ff387a10  1.500000  10.400000
380  ff5ac707  0.800000   2.310345
381  ff635a91  1.000000   7.583333
382  ff7f1779  1.000000  12.833333
383  ff9a11ae  1.952381  19.448276

[384 rows x 3 columns]


In [43]:
def calculate_insertions_per_min(df):

    df_q = df[(df['down_event']=='q')&(df['text_change']=='q')]

    # Convert down_time from milliseconds to seconds
    df_q['down_time_sec'] = df_q['down_time'] / 1000
    # Convert down_time_sec to datetime
    df_q['down_time_sec'] = pd.to_datetime(df_q['down_time_sec'], unit='s')
    # Set down_time_sec as index
    df_q.set_index('down_time_sec', inplace=True)
    # Calculate the length of deletions
    df_q['insert_q'] = df_q['text_change'].str.len()
    # Resample to 1 minute intervals and sum deletion length
    df_sum = df_q.groupby('id').resample('1T')['insert_q'].sum()
    # Calculate the average of the sums for each ID
    df_avg = df_sum.groupby('id').mean()
    # Reset index}
    df_avg = df_avg.reset_index()

    return df_avg

# Process the DataFrames
df_q = calculate_insertions_per_min(df)

df_q

C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\3697091739.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q['down_time_sec'] = df_q['down_time'] / 1000
C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\3697091739.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q['down_time_sec'] = pd.to_datetime(df_q['down_time_sec'], unit='s')
C:\Users\luis.pinto1\AppData\Local\Temp\ipykernel_9184\3697091739.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,id,insert_q
0,001519c8,55.620690
1,0022f953,55.148148
2,0042269b,96.566667
3,0059420b,47.181818
4,0075873a,59.269231
...,...,...
2466,ffb8c745,105.296296
2467,ffbef7e5,62.433333
2468,ffccd6fd,38.760000
2469,ffec5b38,102.652174
